# Process Data from MongoDB from Hamburg Airport

## Imports

In [ ]:
import pandas as pd
import Credentials.credentials as credentials
import pymongo

## Connect to MongoDB

In [ ]:
client = pymongo.MongoClient(
        f"mongodb+srv://moritzzoepffel:{credentials.MONGO_DB_PW}@{credentials.MONGO_DB_DB}/?retryWrites=true&w=majority")
db = client.test

## Get Data from MongoDB

In [ ]:
db.list_collection_names()

In [ ]:
collection_arrivals = db['flights_arrivals']
collection_departures = db['flights_departures']

## Convert to Pandas Dataframe

In [ ]:
df_arrivals = pd.DataFrame(list(collection_arrivals.find()))
df_departures = pd.DataFrame(list(collection_departures.find()))

### Arrivals

In [ ]:
df_arrivals['plannedArrivalTime'] = pd.to_datetime(df_arrivals['plannedArrivalTime'])
df_arrivals['actualArrivalTime'] = pd.to_datetime(df_arrivals['actualArrivalTime'])

In [ ]:
# only arrivals after today
df_arrivals_future = df_arrivals[df_arrivals['plannedArrivalTime'] > pd.Timestamp.today().tz_localize('UTC')]
df_arrivals_past = df_arrivals[df_arrivals['plannedArrivalTime'] < pd.Timestamp.today().tz_localize('UTC')]

In [ ]:
df_arrivals_past.info()

In [ ]:
df_departures_past

In [ ]:
df_departures['plannedDepartureTime'] = pd.to_datetime(df_departures['plannedDepartureTime'])
df_departures['actualDepartureTime'] = pd.to_datetime(df_departures['actualDepartureTime'])
df_departures["year"] = df_departures["plannedDepartureTime"].dt.year
df_departures["month"] = df_departures["plannedDepartureTime"].dt.month
df_departures["day"] = df_departures["plannedDepartureTime"].dt.day
df_departures["hour"] = df_departures["plannedDepartureTime"].dt.hour
df_departures["minute"] = df_departures["plannedDepartureTime"].dt.minute
df_departures["second"] = df_departures["plannedDepartureTime"].dt.second
df_departures["weekday"] = df_departures["plannedDepartureTime"].dt.weekday
df_departures["week"] = df_departures["plannedDepartureTime"].dt.week
df_departures["quarter"] = df_departures["plannedDepartureTime"].dt.quarter

In [ ]:
df_departures_past = df_departures[df_departures['plannedDepartureTime'] < pd.Timestamp.today().tz_localize('UTC')]

In [ ]:
#drop rows that have None in the actualDepartureTime
df_departures_past = df_departures_past.dropna(subset=['actualDepartureTime'])

In [ ]:
df_departures_past.info()

In [ ]:
df_departures_past["delay"] = df_departures_past["actualDepartureTime"] - df_departures_past["plannedDepartureTime"]
df_departures_past["delay"] = df_departures_past["delay"].dt.total_seconds()

In [ ]:
# mean delay per airlineName
mean_delay = pd.DataFrame(df_departures_past.groupby("airlineName")["delay"].mean())

In [ ]:
mean_delay

In [ ]:
df_departures_past = df_departures_past.merge(mean_delay, on="airlineName")

In [ ]:
#rename delay_x to delay and delay_y to mean_delay
df_departures_past = df_departures_past.rename(columns={"delay_x": "delay", "delay_y": "mean_delay"})

In [ ]:
df_departures_past

### Departures

In [ ]:
df_departures

In [ ]:
collection = db['flights_departures_past_cleaned']
collection.insert_many(df_departures_past.to_dict('records'))